In [2]:
import pandas as pd
import regex as re
import fuzzywuzzy
from fuzzywuzzy import fuzz

ModuleNotFoundError: No module named 'fuzzywuzzy'

In [ ]:
df = pd.read_excel('../data/AEPSi_data.xlsx', 2)
df

In [ ]:
#Start just by taking a look at the column
df.Examiner

In [ ]:
#Look at the value counts for the column
df.Examiner.value_counts()

In [ ]:
#First, an easy adjustment, stripping any leading or lagging spaces
df.Examiner = df.Examiner.str.strip()
df.Examiner

In [ ]:
#We do see a decent reduction in the number of unique values (as indicated here by the number of lengths)
df.Examiner.value_counts()

In [ ]:
#Now we'll look for any cells that contain a semicolon and keep only what is to the left of the semicolon
df.Examiner = df.Examiner.str.extract('(^[^;]+)')
df.Examiner

In [ ]:
#This leads to another reduction in unique values
df.Examiner.value_counts()

In [ ]:
#Making the value counts into a dataframe
ex_counts = df.Examiner.value_counts().to_frame()
#Keeping only those entries that occur more than once
ex_counts = ex_counts.loc[ex_counts.Examiner > 1]
#Turning that into a list
Examiner_list =ex_counts.index.to_list()
Examiner_list

In [ ]:
#creating a column to indicate whether or not a name is changed (Y=Yes, N=No)
df['changed'] = 'N'

#Building a for loop that compares the entries in the examiner column to the entries in our list
for name in Examiner_list:
    
        for ind, row in df.iterrows():
            #For any row that indicates it has NOT be changed, the following happens
            if row.changed == 'N':
                #Create a ration of how similar the entry in the dataframe is to the entry in the list
                ratio = fuzz.token_set_ratio(name, row.Examiner)
                #For any entries that have a ratio of 100, meaning they match exactly...
                if ratio == 100:
                    #Go ahead and mark the 'changed' column as Y so that row won't be looked at again
                    df.loc[ind, 'changed'] = 'Y'
                #For  any entries that are very similar (ratio > 88) but not exactly the same...
                if ((ratio > 88)  & (ratio < 100)):
                    #go ahead and change the entry in the dataframe to match the entry in the list
                    df.loc[ind, 'Examiner'] = name
                    #and mark the 'changed' column as Y so that row won't be evaluated again
                    df.loc[ind, 'changed'] = 'Y'
#This loop continues until all rows are changed or have no match that creates a ratio greater than 88            
        

In [ ]:
#Take a look at how many unique values remain
df.Examiner.nunique()

In [ ]:
#look at how many rows were marked Y and N after running our loop
df['changed'].value_counts()

In [ ]:
#look at the updated value counts for Examiner names
df.Examiner.value_counts()

In [ ]:
#Get rid of our 'changed' column now that it's served its purpose
df = df.drop(columns = 'changed')

In [ ]:
#save the cleaned up df to a csv
df.to_csv('../data/cleaned_AEPS.csv', index = False)

In [ ]:
#read in the new csv to check it, note the warning giving you a heads up about some columns with mixed datatypes
new = pd.read_csv('../data/cleaned_AEPS.csv')

In [ ]:
new